In [1]:
#thats for API propose, can been found in the IllustrisTNG example for API
import requests

def get(path, params=None):
    # make HTTP GET request to path
    headers = {"api-key":"83fb2a511946e50d7d8eb5c334bbc58c"}
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [2]:
#libraries
import illustris_python as il
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree
from scipy.interpolate import griddata
from scipy import interpolate

from astropy import constants as const

# Constants
G = const.G.cgs.value  # Gravitational constant in cm^3 g^-1 s^-2
m_p = const.m_p.cgs.value  # Proton mass in g
c = const.c.cgs.value  # Speed of light in cm/s
sigma_T = const.sigma_T.cgs.value  # Thomson cross-section in cm^2

e_r = 0.2 #radiative accretion efficiency

e_fh = 0.05 #high-accretion state 
e_fm = 0.2 #low-accretion state

Msun = 1.989e33 # in grams
Gyr_to_s = 3.15576e16 # in seconds

h = 0.6774 #for illustrisTNG

#Path of the simulation in Jupyterlab
basePath = './sims.illustris/Illustris-1/output/'

#### Galaxy selection

In [3]:
#load the index of Subhalo table of the first (central) Subfind subhalo within this FoF group
GroupFirstSub = il.groupcat.loadHalos(basePath,135,fields=['GroupFirstSub'])
print(GroupFirstSub.dtype, GroupFirstSub.shape)


#load the integer counter of the total number of particles/cells, split into the six different types, in this group.
GroupLenType = il.groupcat.loadHalos(basePath,135,fields=['GroupLenType'])
print(GroupLenType.dtype, GroupLenType.shape)

uint32 (7713601,)
uint32 (7713601, 6)


In [4]:
# Particle type number for black holes
ptNumBH = il.snapshot.partTypeNum('bh')

# Array of number of black holes in each halo
numBH = GroupLenType[:, ptNumBH] #ptNumBH = 5

# Find indices of halos that contain exactly one black hole and at least one subhalo
w = np.where((numBH == 1) & (GroupFirstSub >= 0))[0]

# Find the first subhalo in each of these halos
galaxies = GroupFirstSub[w]

len(galaxies)

21455

In [5]:
#ids of 10 most massive galaxies
ids = galaxies[0:10]
ids

array([305959, 312924, 321863, 322384, 324170, 334336, 337748, 345367,
       345729, 347122], dtype=uint32)

Check the galaxies if the have supermassive black hole

In [6]:
for id in ids:
    url = "http://www.tng-project.org/api/Illustris-1/snapshots/z=0/subhalos/" + str(id)
    subhalo = get(url)
    print(id, np.log10(subhalo['mass_bhs'] *((1e10) / h)), subhalo['len_bhs'])

305959 8.778439658461915 1
312924 8.74446968827434 1
321863 8.830844473012226 1
322384 8.527146794943475 1
324170 8.553454446999773 1
334336 8.400088845975754 1
337748 8.439903497920138 1
345367 8.41405014138325 1
345729 8.675091232257937 1
347122 8.597645965866906 1


### Make a CSV file! 
##### With the data that we want to use in order to make the graphs/diagrams for galaxy id=305959 which is the most massive galaxy with one black hole (SMBH)

In [3]:
ids = [305959] #galaxy id
target_snaps = [135, 103, 85, 68, 60, 54]  # list of target snapshots(redshifts) here 

# the target snapshots list are the 6 out the 10 full snapshots that illustris-1 have 
# 135 snapshots corresponds to z=0 and the 54 snapshot to z=4

#this code. tries to find the galaxies in previous redshifts
progenitor_ids_list = []

for id in ids:
    id_progenitors = []
    start_url = "http://www.tng-project.org/api/Illustris-1/snapshots/z=0/subhalos/" + str(id)
    sub = get(start_url)
    current_snap = 135

    for snap in target_snaps:
        while current_snap > snap:
            # request the full subhalo details of the progenitor by following the sublink URL
            try:
                sub = get(sub['related']['sublink_progenitor'])
                current_snap = sub['snap']
            except KeyError:  # KeyError occurs if there's no progenitor
                break  # exit the while loop and go to the next snap

        if current_snap == snap:  # if the snapshot has a progenitor
            id_progenitors.append(sub['id'])
            print(f'Progenitor of {id} at snap={snap} is {sub["id"]}')
        else:
            print(f'Progenitor of {id} not followed to snap={snap}!')
            id_progenitors.append(-1)

    progenitor_ids_list.append(id_progenitors)

# Transpose the list to match the target_snap order
progenitor_ids_list = list(map(list, zip(*progenitor_ids_list)))

# Exclude -1 values
progenitor_ids_list = [sublist for sublist in progenitor_ids_list if sublist != [-1]*len(ids)]

# Flatten the list
progenitor_ids_list = [id for sublist in progenitor_ids_list for id in sublist]

# Reverse the list
progenitor_ids_list.reverse()

print(progenitor_ids_list)

Progenitor of 305959 at snap=135 is 305959
Progenitor of 305959 at snap=103 is 192798
Progenitor of 305959 at snap=85 is 195558
Progenitor of 305959 at snap=68 is 106855
Progenitor of 305959 at snap=60 is 68043
Progenitor of 305959 at snap=54 is 55141
[55141, 68043, 106855, 195558, 192798, 305959]


In [4]:
# target snapshots
snapshots = [54, 60, 68, 85, 103, 135]
print(snapshots)
#below are the ids that were found from above
ids = progenitor_ids_list
print(ids)

[54, 60, 68, 85, 103, 135]
[55141, 68043, 106855, 195558, 192798, 305959]


In [9]:
#Redshift are stored in the list `redshift`
redshift = [4, 3, 2, 1, 0.5, 0]

# Create a DataFrame from your data
df = pd.DataFrame({
    'Redshift (z)': redshift
})

# Write the DataFrame to a CSV file
df.to_csv('ILL_305959.csv', index=False)

In [10]:
# You can view the first few rows of the DataFrame
df = pd.read_csv('ILL_305959.csv')
print(df.head())

   Redshift (z)
0           4.0
1           3.0
2           2.0
3           1.0
4           0.5


In [9]:
#fuction that calculate's the divergence of the gas relative to the BH position in the galaxy
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adhust as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    # Load the particle data for the subhalo
    part_data = il.snapshot.loadSubhalo(basePath, snap, i, partType='gas', fields=['Coordinates', 'Velocities'])
    
    agn_position = il.groupcat.loadSingle(basePath, snap, subhaloID=i)
    agn_position = agn_position['SubhaloPos']

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

    # Print the average divergence
    print(f'Average divergence at snap={snap}, id={i}: {average_divergence}')
#read the file    
df = pd.read_csv('ILL_305959.csv')
#append them to the new column 
df['Average divergence'] = avg_divergence_list
df.to_csv('ILL_305959.csv', index=False)

Average divergence at snap=54, id=55141: -10.540335697194278
Average divergence at snap=60, id=68043: -9.290087654976606
Average divergence at snap=68, id=106855: -4.891973155221894
Average divergence at snap=85, id=195558: -4.0501670977844
Average divergence at snap=103, id=192798: 0.4099557622048855
Average divergence at snap=135, id=305959: 5.580304564829707


In [10]:
#fuction that calculate's the divergence of the gas relative to the BH position in the Halo
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adjusted as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and their snapshot
for snap, i in zip(snapshots, ids):
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snap, subhaloID=i)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snap, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snap, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

    # Print the average divergence
    print(f'Average divergence at snap={snap}, galaxy id={i}, halo id={main_halo_id}: {average_divergence}')
#read the file    
df = pd.read_csv('ILL_305959.csv')
#append them to the new column 
df['Average divergence of Halo'] = avg_divergence_list
df.to_csv('ILL_305959.csv', index=False)

Average divergence at snap=54, galaxy id=55141, halo id=1653: -11.70368749269247
Average divergence at snap=60, galaxy id=68043, halo id=974: -9.278331955267847
Average divergence at snap=68, galaxy id=106855, halo id=697: -4.036630663581051
Average divergence at snap=85, galaxy id=195558, halo id=629: -4.127504497479796
Average divergence at snap=103, galaxy id=192798, halo id=208: 7.482354996800005
Average divergence at snap=135, galaxy id=305959, halo id=272: 4.642933775007915


In [6]:
#fuction that calculate's the radial velocity of the gas relative to the BH position in the galaxy
def calculate_radial_velocity(positions, velocities, agn_position):
    # Calculate the relative positions and distances of the particles
    relative_positions = positions - agn_position
    distances = np.linalg.norm(relative_positions, axis=1)

    # Calculate the radial velocities
    radial_velocities = np.sum(relative_positions * velocities, axis=1) / distances

    return radial_velocities

# Define lists to store the results
average_radial_velocity_list = []


# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snap, subhaloID=i)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snap, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snap, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the radial velocities
    radial_velocities = calculate_radial_velocity(part_data['Coordinates'], part_data['Velocities'], agn_position)

    # Calculate the average radial velocity
    average_radial_velocity = np.mean(radial_velocities)
    
    average_radial_velocity_list.append(average_radial_velocity)

    # Print the average radial velocity
    print(f'Average radial velocity at snap={snap}, id={i}, halo id={main_halo_id}: {average_radial_velocity}')
    
df = pd.read_csv('ILL_305959.csv')
df['Average radial velocity'] = average_radial_velocity_list
df.to_csv('ILL_305959.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


Average radial velocity at snap=54, id=55141, halo id=1653: nan
Average radial velocity at snap=60, id=68043, halo id=974: -230.07628750708975
Average radial velocity at snap=68, id=106855, halo id=697: -97.78975728792986
Average radial velocity at snap=85, id=195558, halo id=629: -69.78676057960624
Average radial velocity at snap=103, id=192798, halo id=208: -48.746116667697294
Average radial velocity at snap=135, id=305959, halo id=272: 66.65129063029593


In [13]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapshots, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    print(f'Blackhole Masses at snap={snap}, id={id}: {bh_mass}')
    
df = pd.read_csv('ILL_305959.csv')
df['Blackhole Masses'] = bhmass_values
df.to_csv('ILL_305959.csv', index=False)

Blackhole Masses at snap=54, id=55141: 5.524257081657102
Blackhole Masses at snap=60, id=68043: 6.497566912618314
Blackhole Masses at snap=68, id=106855: 7.211136649140027
Blackhole Masses at snap=85, id=195558: 7.351369191241512
Blackhole Masses at snap=103, id=192798: 7.972280368105746
Blackhole Masses at snap=135, id=305959: 8.778437294319545


In [14]:
# List to store BHmdot
bhmdot_values = []

fields = 'BH_Mdot'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapshots, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BHmdot.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_dot = bh_data[0] * ((1e10 * Msun) / (h * Gyr_to_s)) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mdot' in bh_data and len(bh_data['BH_Mdot']) > 0:  # Check if there is BH data.
        bh_dot = bh_data['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  # If no BHmdot, set the BHmdot to zero.
        bh_dot = 0.0
        
    #covert the BHmdot to log10 
    bh_dot = np.log10(bh_dot)

    # Append the BHmdot to the list.
    bhmdot_values.append(bh_dot)
    
    
# Print the BHmdot
print("Bhmdot:")
print(bhmdot_values) 

df = pd.read_csv('ILL_305959.csv')
df['Bhmdot'] = bhmdot_values
df.to_csv('ILL_305959.csv', index=False)

Bhmdot:
[22.71530800944891, 24.484097204982703, 21.329509413572435, 19.9586552074289, 24.725514656953813, 24.427875661216024]


In [15]:
# List to store Star formation rate
sfr_values = []

# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    #load the galaxies
    sfr = il.groupcat.loadSingle(basePath, snap, subhaloID=i)
    #extract the value of the sfr
    sfr_value = sfr['SubhaloSFRinHalfRad']
    sfr_values.append(sfr_value)
    print(f'Subhalo SFR at snap={snap}, id={i}: {sfr_value}')

df = pd.read_csv('ILL_305959.csv')
df['SFR'] = sfr_values
df.to_csv('ILL_305959.csv', index=False)

Subhalo SFR at snap=54, id=55141: 1.874635934829712
Subhalo SFR at snap=60, id=68043: 7.331963539123535
Subhalo SFR at snap=68, id=106855: 11.120349884033203
Subhalo SFR at snap=85, id=195558: 11.676556587219238
Subhalo SFR at snap=103, id=192798: 34.291831970214844
Subhalo SFR at snap=135, id=305959: 0.4768887162208557


In [16]:
# Create empty list to store Eddington rate values
λ_rate_list = []

fields=['BH_Mass', 'BH_Mdot']

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    if blackholes and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = 0.0  # Set to np.nan so that the resulting λ_rate will be np.nan.
        
    if blackholes and 'BH_Mass' in blackholes and len(blackholes['BH_Mass']) > 0 and blackholes['BH_Mass'][0] != 0: 
        bh_mass = blackholes['BH_Mass'][0] * ((1e10* Msun) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0

    # Calculate L_bol and L_edd
    L_bol = e_r * bhdot * c**2
    
    L_edd = 4 * np.pi * G * bh_mass * m_p * c / sigma_T
        
    # Calculate λ_rate
    λ_rate = (L_bol / L_edd) if L_edd != 0 else 0

    # Append λ_rate to the list
    λ_rate_list.append(λ_rate)

# Print Eddington rate values
print("Eddington rate λ values", λ_rate_list)

df = pd.read_csv('ILL_305959.csv')
df['λ'] = λ_rate_list
df.to_csv('ILL_305959.csv', index=False)

Eddington rate λ values [0.2219400398908138, 1.3858461649060991, 0.00018774029519565033, 5.787230072257482e-06, 0.08098769167630993, 0.006377098114638177]


In [17]:
# Create empty list to store the bolometric luminosity values
L_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting L_bol will be np.nan.

    # Calculate L_bol
    L_bol = e_r * bhdot * c**2

    
    # Append bolometric luminosity to the list
    L_list.append(np.log10(L_bol))


# Print bolometric luminosity values
print("Bol Luminosity values:")
print(L_list)

df = pd.read_csv('ILL_305959.csv')
df['L_bol'] = L_list
df.to_csv('ILL_305959.csv', index=False)

Bol Luminosity values:
[42.96997941096875, 44.73876860650254, 41.584180815092274, 40.21332660894874, 44.98018605847365, 44.682547062735864]


In [18]:
# Create empty list to store the Eddington luminosity values
L_edd_list = []

e_r = 0.2

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    if blackholes and 'BH_Mass' in blackholes and len(blackholes['BH_Mass']) > 0 and blackholes['BH_Mass'][0] != 0: 
        bh_mass = blackholes['BH_Mass'][0] * ((1e10* Msun) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0

    # Calculate L_edd
    L_edd = 4 * np.pi * G * bh_mass * m_p * c / sigma_T

    
    # Append Eddington luminosity to the list
    L_edd_list.append(np.log10(L_edd))


# Print Eddington luminosity values
print("Edd Luminosity values:")
print(L_edd_list)

df = pd.read_csv('ILL_305959.csv')
df['L_edd'] = L_edd_list
df.to_csv('ILL_305959.csv', index=False)

Edd Luminosity values:
[43.62374375121096, 44.59705358217217, 45.310623318693885, 45.45085586079537, 46.071767037659605, 46.87792396387341]


In [19]:
# Create empty list to store the feedback energy of the AGN
E_feed_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting feedback energy will be np.nan.

        
    # Calculate feedback energy
    E_feed = e_fm * e_r * bhdot * c**2

    
    # Append feedback energy to the list
    E_feed_list.append(np.log10(E_feed))


# Print feedback energy values
print("E_feed values:")
print(E_feed_list)

df = pd.read_csv('ILL_305959.csv')
df['E_feed'] = E_feed_list
df.to_csv('ILL_305959.csv', index=False)

E_feed values:
[42.27100940663273, 44.03979860216652, 40.885210810756256, 39.51435660461272, 44.28121605413763, 43.983577058399845]


In [20]:
# Prepare empty lists to store the BH_CumEgyInjection_QM
QM_values = []

# Loop over each subhalo ID and their snapshot
for snapshot, id in zip(snapshots, ids):
    # Load necessary fields
    fields = ['BH_CumEgyInjection_QM']
    data = il.snapshot.loadSubhalo(basePath, snapshot, id, 'BH', fields=fields)
    
    # Check if 'BH_CumEgyInjection_QM' exists in the data dictionary and if it's not empty
    if 'BH_CumEgyInjection_QM' in data and len(data['BH_CumEgyInjection_QM']) > 0:
        # Convert units
        BH_CumEgyInjection_QM_phys = np.sqrt(data['BH_CumEgyInjection_QM'][0] * 1e10 / 0.7 / (0.978 / 0.7)**2)

        # Calculate the BH_CumEgyInjection_QM and replace -inf with 0
        QM_log = np.log10(BH_CumEgyInjection_QM_phys)

        # Replace -inf with 0
        if np.isneginf(QM_log):
            QM_log = 0

        QM_values.append(QM_log)

        print(f'Logarithmic QM at snap={snapshot}, id={id}: {QM_log}')
    else:
        print(f'No BH_CumEgyInjection_QM data for snap={snapshot}, id={id}')

No BH_CumEgyInjection_QM data for snap=54, id=55141
No BH_CumEgyInjection_QM data for snap=60, id=68043
No BH_CumEgyInjection_QM data for snap=68, id=106855
No BH_CumEgyInjection_QM data for snap=85, id=195558
No BH_CumEgyInjection_QM data for snap=103, id=192798
No BH_CumEgyInjection_QM data for snap=135, id=305959


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # This is added back by InteractiveShellApp.init_path()


In [21]:
#lets view the compilation of the file 
df = pd.read_csv('ILL_305959.csv')
print(df.head())

   Redshift (z)  Average divergence  Average radial velocity  \
0           4.0          -10.594588                      NaN   
1           3.0           -9.338459              -181.328333   
2           2.0           -4.977905               -70.544716   
3           1.0           -4.051704               -66.025966   
4           0.5            0.375005               -19.598002   

   Blackhole Masses     Bhmdot        SFR         λ      L_bol      L_edd  \
0          5.524257  22.715308   1.874636  0.221940  42.969979  43.623744   
1          6.497567  24.484097   7.331964  1.385846  44.738769  44.597054   
2          7.211137  21.329509  11.120350  0.000188  41.584181  45.310623   
3          7.351369  19.958655  11.676557  0.000006  40.213327  45.450856   
4          7.972280  24.725515  34.291832  0.080988  44.980186  46.071767   

      E_feed  
0  42.271009  
1  44.039799  
2  40.885211  
3  39.514357  
4  44.281216  


### Next galaxy is id=321863

##### same process as the above

In [7]:
ids = [321863] #galaxy id
target_snaps = [135, 103, 85, 68, 60, 54]  # list of target snapshots here
# the target snapshots list are the 7 out the 10 full snapshots that illustrisTNG have 
# 135 snapshots corresponds to z=0 and the 54 snapshot to z=4

progenitor_ids_list = []

for id in ids:
    id_progenitors = []
    start_url = "http://www.tng-project.org/api/Illustris-1/snapshots/z=0/subhalos/" + str(id)
    sub = get(start_url)
    current_snap = 135

    for snap in target_snaps:
        while current_snap > snap:
            # request the full subhalo details of the progenitor by following the sublink URL
            try:
                sub = get(sub['related']['sublink_progenitor'])
                current_snap = sub['snap']
            except KeyError:  # KeyError occurs if there's no progenitor
                break  # exit the while loop and go to the next snap

        if current_snap == snap:  # if the snapshot has a progenitor
            id_progenitors.append(sub['id'])
            print(f'Progenitor of {id} at snap={snap} is {sub["id"]}')
        else:
            print(f'Progenitor of {id} not followed to snap={snap}!')
            id_progenitors.append(-1)

    progenitor_ids_list.append(id_progenitors)

# Transpose the list to match the target_snap order
progenitor_ids_list = list(map(list, zip(*progenitor_ids_list)))

# Exclude -1 values
progenitor_ids_list = [sublist for sublist in progenitor_ids_list if sublist != [-1]*len(ids)]

# Flatten the list
progenitor_ids_list = [id for sublist in progenitor_ids_list for id in sublist]

# Reverse the list
progenitor_ids_list.reverse()

print(progenitor_ids_list)

Progenitor of 321863 at snap=135 is 321863
Progenitor of 321863 at snap=103 is 218780
Progenitor of 321863 at snap=85 is 161702
Progenitor of 321863 at snap=68 is 71201
Progenitor of 321863 at snap=60 is 48081
Progenitor of 321863 at snap=54 is 55206
[55206, 48081, 71201, 161702, 218780, 321863]


In [8]:
# target snapshots
snapshots = [54, 60, 68, 85, 103, 135]
print(snapshots)
#below are the ids that were found from above
ids = progenitor_ids_list
print(ids)

[54, 60, 68, 85, 103, 135]
[55206, 48081, 71201, 161702, 218780, 321863]


In [24]:
#Redshift are stored in the list `redshift`
redshift = [4, 3, 2, 1, 0.5, 0]

# Create a DataFrame from your data
df = pd.DataFrame({
    'Redshift (z)': redshift
})

# Write the DataFrame to a CSV file
df.to_csv('ILL_321863.csv', index=False)

In [13]:
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adhust as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    # Load the particle data for the subhalo
    part_data = il.snapshot.loadSubhalo(basePath, snap, i, partType='gas', fields=['Coordinates', 'Velocities'])
    
    agn_position = il.groupcat.loadSingle(basePath, snap, subhaloID=i)
    agn_position = agn_position['SubhaloPos']

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

    # Print the average divergence
    print(f'Average divergence at snap={snap}, id={i}: {average_divergence}')
#read the file    
df = pd.read_csv('ILL_321863.csv')
#append them to the new column 
df['Average divergence'] = avg_divergence_list
df.to_csv('ILL_321863.csv', index=False)

Average divergence at snap=54, id=55206: -6.96751112986803
Average divergence at snap=60, id=48081: -3.1101808842374234
Average divergence at snap=68, id=71201: -4.896644450462997
Average divergence at snap=85, id=161702: 6.875333286773741
Average divergence at snap=103, id=218780: 0.23513987167227268
Average divergence at snap=135, id=321863: 4.54593638033104


In [14]:
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adjusted as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and their snapshot
for snap, i in zip(snapshots, ids):
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snap, subhaloID=i)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snap, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snap, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

    # Print the average divergence of Halo
    print(f'Average divergence at snap={snap}, galaxy id={i}, halo id={main_halo_id}: {average_divergence}')
#read the file    
df = pd.read_csv('ILL_321863.csv')
#append them to the new column 
df['Average divergence of Halo'] = avg_divergence_list
df.to_csv('ILL_321863.csv', index=False)

Average divergence at snap=54, galaxy id=55206, halo id=1656: -7.244204299779538
Average divergence at snap=60, galaxy id=48081, halo id=502: -6.0289983963879346
Average divergence at snap=68, galaxy id=71201, halo id=288: -4.101717636060924
Average divergence at snap=85, galaxy id=161702, halo id=365: 13.602447231365383
Average divergence at snap=103, galaxy id=218780, halo id=313: 1.4799145326639116
Average divergence at snap=135, galaxy id=321863, halo id=344: 13.55854560648942


In [9]:
def calculate_radial_velocity(positions, velocities, agn_position):
    # Calculate the relative positions and distances of the particles
    relative_positions = positions - agn_position
    distances = np.linalg.norm(relative_positions, axis=1)

    # Calculate the radial velocities
    radial_velocities = np.sum(relative_positions * velocities, axis=1) / distances

    return radial_velocities

# Define lists to store the results
average_radial_velocity_list = []


# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snap, subhaloID=i)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snap, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snap, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the radial velocities
    radial_velocities = calculate_radial_velocity(part_data['Coordinates'], part_data['Velocities'], agn_position)

    # Calculate the average radial velocity
    average_radial_velocity = np.mean(radial_velocities)
    
    average_radial_velocity_list.append(average_radial_velocity)

    # Print the average radial velocity
    print(f'Average radial velocity at snap={snap}, id={i}, halo id={main_halo_id}: {average_radial_velocity}')
    
df = pd.read_csv('ILL_321863.csv')
df['Average radial velocity'] = average_radial_velocity_list
df.to_csv('ILL_321863.csv', index=False)

Average radial velocity at snap=54, id=55206, halo id=1656: -80.94949211664127
Average radial velocity at snap=60, id=48081, halo id=502: 27.519725895891664
Average radial velocity at snap=68, id=71201, halo id=288: -284.19106285513175
Average radial velocity at snap=85, id=161702, halo id=365: 138.06909126845326
Average radial velocity at snap=103, id=218780, halo id=313: -295.0325580358673
Average radial velocity at snap=135, id=321863, halo id=344: 149.0458912850704


In [27]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapshots, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    print(f'Blackhole Masses at snap={snap}, id={id}: {bh_mass}')
    
df = pd.read_csv('ILL_321863.csv')
df['Blackhole Masses'] = bhmass_values
df.to_csv('ILL_321863.csv', index=False)

Blackhole Masses at snap=54, id=55206: 6.544635847694498
Blackhole Masses at snap=60, id=48081: 6.890266536255159
Blackhole Masses at snap=68, id=71201: 7.434814584940885
Blackhole Masses at snap=85, id=161702: 8.50232927968103
Blackhole Masses at snap=103, id=218780: 8.612044567646167
Blackhole Masses at snap=135, id=321863: 8.830848571021425


In [28]:
# List to store BHmdot
bhmdot_values = []

fields = 'BH_Mdot'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapshots, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BHmdot.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_dot = bh_data[0] * ((1e10 * Msun) / (h * Gyr_to_s)) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mdot' in bh_data and len(bh_data['BH_Mdot']) > 0:  # Check if there is BH data.
        bh_dot = bh_data['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  # If no BHmdot data, set the BHmdot to zero.
        bh_dot = 0.0
        
    #covert the BHmdot to log10 
    bh_dot = np.log10(bh_dot)

    # Append the BHmdot to the list.
    bhmdot_values.append(bh_dot)
    
    
# Print the BHmdot
print("Bhmdot:")
print(bhmdot_values) 

df = pd.read_csv('ILL_321863.csv')
df['Bhmdot'] = bhmdot_values
df.to_csv('ILL_321863.csv', index=False)

Bhmdot:
[23.687527092330058, 23.52077757202463, 23.954258820839932, 23.03122237853417, 24.157074843596273, 16.562664104896324]


In [29]:
# List to store Star formation rate
sfr_values = []

# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    #load the galaxies
    sfr = il.groupcat.loadSingle(basePath, snap, subhaloID=i)
    #extract the value of the sfr
    sfr_value = sfr['SubhaloSFRinHalfRad']
    sfr_values.append(sfr_value)
    print(f'Subhalo SFR at snap={snap}, id={i}: {sfr_value}')

df = pd.read_csv('ILL_321863.csv')
df['SFR'] = sfr_values
df.to_csv('ILL_321863.csv', index=False)

Subhalo SFR at snap=54, id=55206: 3.0659966468811035
Subhalo SFR at snap=60, id=48081: 9.99066162109375
Subhalo SFR at snap=68, id=71201: 34.09093475341797
Subhalo SFR at snap=85, id=161702: 3.7414474487304688
Subhalo SFR at snap=103, id=218780: 0.8996496200561523
Subhalo SFR at snap=135, id=321863: 0.0


In [30]:
# Create empty list to store Eddington rate values
λ_rate_list = []

fields=['BH_Mass', 'BH_Mdot']

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    if blackholes and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = 0.0  # Set to np.nan so that the resulting λ_rate will be np.nan.
        
    if blackholes and 'BH_Mass' in blackholes and len(blackholes['BH_Mass']) > 0 and blackholes['BH_Mass'][0] != 0: 
        bh_mass = blackholes['BH_Mass'][0] * ((1e10* Msun) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0

    # Calculate L_bol and L_edd
    L_bol = e_r * bhdot * c**2
    
    L_edd = 4 * np.pi * G * bh_mass * m_p * c / sigma_T
        
    # Calculate λ_rate
    λ_rate = (L_bol / L_edd) if L_edd != 0 else 0

    # Append λ_rate to the list
    λ_rate_list.append(λ_rate)

# Print Eddington rate values
print("Eddington rate λ values", λ_rate_list)

df = pd.read_csv('ILL_321863.csv')
df['λ'] = λ_rate_list
df.to_csv('ILL_321863.csv', index=False)

Eddington rate λ values [0.19864424003403272, 0.06105141972531319, 0.0472747202820977, 0.000483145379771978, 0.0050143541983722765, 7.709039409509626e-11]


In [31]:
# Create empty list to store the bolometric luminosity values
L_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting L_bol will be np.nan.

    # Calculate L_bol
    L_bol = e_r * bhdot * c**2

    
    # Append bolometric luminosity to the list
    L_list.append(np.log10(L_bol))


# Print bolometric luminosity values
print("Bol Luminosity values:")
print(L_list)

df = pd.read_csv('ILL_321863.csv')
df['L_bol'] = L_list
df.to_csv('ILL_321863.csv', index=False)

Bol Luminosity values:
[43.942198493849894, 43.77544897354447, 44.20893022235977, 43.28589378005401, 44.41174624511611, 36.817335506416164]


In [32]:
# Create empty list to store the Eddington luminosity values
L_edd_list = []

e_r = 0.2

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    if blackholes and 'BH_Mass' in blackholes and len(blackholes['BH_Mass']) > 0 and blackholes['BH_Mass'][0] != 0: 
        bh_mass = blackholes['BH_Mass'][0] * ((1e10* Msun) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0

    # Calculate L_edd
    L_edd = 4 * np.pi * G * bh_mass * m_p * c / sigma_T

    
    # Append Eddington luminosity to the list
    L_edd_list.append(np.log10(L_edd))


# Print Eddington luminosity values
print("Edd Luminosity values:")
print(L_edd_list)

df = pd.read_csv('ILL_321863.csv')
df['L_edd'] = L_edd_list
df.to_csv('ILL_321863.csv', index=False)

Edd Luminosity values:
[44.644122517248356, 44.989753205809016, 45.53430125449474, 46.60181594923489, 46.71153123720003, 46.93033524057528]


In [33]:
# Create empty list to store the feedback energy of the AGN values
E_feed_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting feedback energy will be np.nan.

        
    # Calculate feedback energy
    E_feed = e_fm * e_r * bhdot * c**2

    
    # Append feedback energy to the list
    E_feed_list.append(np.log10(E_feed))


# Print feedback energy values
print("E_feed values:")
print(E_feed_list)

df = pd.read_csv('ILL_321863.csv')
df['E_feed'] = E_feed_list
df.to_csv('ILL_321863.csv', index=False)

E_feed values:
[43.243228489513875, 43.07647896920845, 43.50996021802375, 42.58692377571799, 43.712776240780094, 36.118365502080145]


In [34]:
#lets view the compilation of the file 
df = pd.read_csv('ILL_321863.csv')
print(df.head())

   Redshift (z)  Average divergence  Average radial velocity  \
0           4.0           -7.283252               -80.921980   
1           3.0           -3.258640                42.515649   
2           2.0           -4.933075              -218.486371   
3           1.0            6.829896               152.274896   
4           0.5            0.220408               -92.871559   

   Blackhole Masses     Bhmdot        SFR         λ      L_bol      L_edd  \
0          6.544636  23.687527   3.065997  0.198644  43.942198  44.644123   
1          6.890267  23.520778   9.990662  0.061051  43.775449  44.989753   
2          7.434815  23.954259  34.090935  0.047275  44.208930  45.534301   
3          8.502329  23.031222   3.741447  0.000483  43.285894  46.601816   
4          8.612045  24.157075   0.899650  0.005014  44.411746  46.711531   

      E_feed  
0  43.243228  
1  43.076479  
2  43.509960  
3  42.586924  
4  43.712776  


### Next galaxy is id=347122

##### same process as the above

In [10]:
ids = [347122] #galaxy id
target_snaps = [135, 103, 85, 68, 60, 54]  # list of target snapshots here
# the target snapshots list are the 7 out the 10 full snapshots that illustrisTNG have 
# 135 snapshots corresponds to z=0 and the 54 snapshot to z=4

progenitor_ids_list = []

for id in ids:
    id_progenitors = []
    start_url = "http://www.tng-project.org/api/Illustris-1/snapshots/z=0/subhalos/" + str(id)
    sub = get(start_url)
    current_snap = 135

    for snap in target_snaps:
        while current_snap > snap:
            # request the full subhalo details of the progenitor by following the sublink URL
            try:
                sub = get(sub['related']['sublink_progenitor'])
                current_snap = sub['snap']
            except KeyError:  # KeyError occurs if there's no progenitor
                break  # exit the while loop and go to the next snap

        if current_snap == snap:  # if the snapshot has a progenitor
            id_progenitors.append(sub['id'])
            print(f'Progenitor of {id} at snap={snap} is {sub["id"]}')
        else:
            print(f'Progenitor of {id} not followed to snap={snap}!')
            id_progenitors.append(-1)

    progenitor_ids_list.append(id_progenitors)

# Transpose the list to match the target_snap order
progenitor_ids_list = list(map(list, zip(*progenitor_ids_list)))

# Exclude -1 values
progenitor_ids_list = [sublist for sublist in progenitor_ids_list if sublist != [-1]*len(ids)]

# Flatten the list
progenitor_ids_list = [id for sublist in progenitor_ids_list for id in sublist]

# Reverse the list
progenitor_ids_list.reverse()

print(progenitor_ids_list)

Progenitor of 347122 at snap=135 is 347122
Progenitor of 347122 at snap=103 is 252489
Progenitor of 347122 at snap=85 is 197241
Progenitor of 347122 at snap=68 is 81102
Progenitor of 347122 at snap=60 is 37546
Progenitor of 347122 at snap=54 is 21803
[21803, 37546, 81102, 197241, 252489, 347122]


In [11]:
# target snapshots
snapshots = [54, 60, 68, 85, 103, 135]
print(snapshots)
#below are the ids that were found from above
ids = progenitor_ids_list
print(ids)

[54, 60, 68, 85, 103, 135]
[21803, 37546, 81102, 197241, 252489, 347122]


In [37]:
#Redshift are stored in the list `redshift`
redshift = [4, 3, 2, 1, 0.5, 0]

# Create a DataFrame from your data
df = pd.DataFrame({
    'Redshift (z)': redshift
})

# Write the DataFrame to a CSV file
df.to_csv('ILL_347122.csv', index=False)

In [17]:
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adhust as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    # Load the particle data for the subhalo
    part_data = il.snapshot.loadSubhalo(basePath, snap, i, partType='gas', fields=['Coordinates', 'Velocities'])
    
    agn_position = il.groupcat.loadSingle(basePath, snap, subhaloID=i)
    agn_position = agn_position['SubhaloPos']

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

    # Print the average divergence
    print(f'Average divergence at snap={snap}, id={i}: {average_divergence}')
#read the file    
df = pd.read_csv('ILL_347122.csv')
#append them to the new column 
df['Average divergence'] = avg_divergence_list
df.to_csv('ILL_347122.csv', index=False)

Average divergence at snap=54, id=21803: -6.44623391704116
Average divergence at snap=60, id=37546: -3.841097358249396
Average divergence at snap=68, id=81102: -0.9402080104565025
Average divergence at snap=85, id=197241: -4.368203660908699
Average divergence at snap=103, id=252489: 8.824853761133046
Average divergence at snap=135, id=347122: 0.18881388621401787


In [18]:
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adjusted as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and their snapshot
for snap, i in zip(snapshots, ids):
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snap, subhaloID=i)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snap, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snap, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

    # Print the average divergence
    print(f'Average divergence at snap={snap}, galaxy id={i}, halo id={main_halo_id}: {average_divergence}')
#read the file    
df = pd.read_csv('ILL_347122.csv')
#append them to the new column 
df['Average divergence of Halo'] = avg_divergence_list
df.to_csv('ILL_347122.csv', index=False)

Average divergence at snap=54, galaxy id=21803, halo id=316: -12.254922529696872
Average divergence at snap=60, galaxy id=37546, halo id=316: -3.646182159765333
Average divergence at snap=68, galaxy id=81102, halo id=380: -0.577607589014709
Average divergence at snap=85, galaxy id=197241, halo id=646: -4.622306223710537
Average divergence at snap=103, galaxy id=252489, halo id=524: 15.679777889895588
Average divergence at snap=135, galaxy id=347122, halo id=494: -0.8461036207897514


In [12]:
def calculate_radial_velocity(positions, velocities, agn_position):
    # Calculate the relative positions and distances of the particles
    relative_positions = positions - agn_position
    distances = np.linalg.norm(relative_positions, axis=1)

    # Calculate the radial velocities
    radial_velocities = np.sum(relative_positions * velocities, axis=1) / distances

    return radial_velocities

# Define lists to store the results
average_radial_velocity_list = []


# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snap, subhaloID=i)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snap, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snap, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the radial velocities
    radial_velocities = calculate_radial_velocity(part_data['Coordinates'], part_data['Velocities'], agn_position)

    # Calculate the average radial velocity
    average_radial_velocity = np.mean(radial_velocities)
    
    average_radial_velocity_list.append(average_radial_velocity)

    # Print the average radial velocity
    print(f'Average radial velocity at snap={snap}, id={i}, halo id={main_halo_id}: {average_radial_velocity}')
    
df = pd.read_csv('ILL_347122.csv')
df['Average radial velocity'] = average_radial_velocity_list
df.to_csv('ILL_347122.csv', index=False)

Average radial velocity at snap=54, id=21803, halo id=316: -117.45271295659464
Average radial velocity at snap=60, id=37546, halo id=316: -161.9850355641179
Average radial velocity at snap=68, id=81102, halo id=380: -110.14259904952058
Average radial velocity at snap=85, id=197241, halo id=646: -22.81455021000121
Average radial velocity at snap=103, id=252489, halo id=524: -51.28775805517962
Average radial velocity at snap=135, id=347122, halo id=494: -36.103885342898366


In [40]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapshots, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    print(f'Blackhole Masses at snap={snap}, id={id}: {bh_mass}')
    
df = pd.read_csv('ILL_347122.csv')
df['Blackhole Masses'] = bhmass_values
df.to_csv('ILL_347122.csv', index=False)

Blackhole Masses at snap=54, id=21803: 6.723656159219463
Blackhole Masses at snap=60, id=37546: 7.656816910595308
Blackhole Masses at snap=68, id=81102: 8.27996336076211
Blackhole Masses at snap=85, id=197241: 8.372110455916253
Blackhole Masses at snap=103, id=252489: 8.493090537757562
Blackhole Masses at snap=135, id=347122: 8.597647335053157


In [41]:
# List to store BHmdot
bhmdot_values = []

fields = 'BH_Mdot'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapshots, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BHmdot.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_dot = bh_data[0] * ((1e10 * Msun) / (h * Gyr_to_s)) #covert the BHmdot
    elif isinstance(bh_data, dict) and 'BH_Mdot' in bh_data and len(bh_data['BH_Mdot']) > 0:  # Check if there is BH data.
        bh_dot = bh_data['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  # If no BHmdot data, set the BHmdot to zero.
        bh_dot = 0.0
        
    #covert the BHmdot to log10 
    bh_dot = np.log10(bh_dot)

    # Append the BHmdot to the list.
    bhmdot_values.append(bh_dot)
    
    
# Print the BHmdot
print("Bhmdot:")
print(bhmdot_values) 

df = pd.read_csv('ILL_347122.csv')
df['Bhmdot'] = bhmdot_values
df.to_csv('ILL_347122.csv', index=False)

Bhmdot:
[24.371712281387207, 23.86099294265594, 23.818116675377517, 24.352828458083476, 24.678302314215745, 23.347276409708716]


In [42]:
# List to store Star formation rate
sfr_values = []

# Loop over each subhalo ID and there snapshot
for snap, i in zip(snapshots, ids):
    #load the galaxies
    sfr = il.groupcat.loadSingle(basePath, snap, subhaloID=i)
    #extract the value of the sfr
    sfr_value = sfr['SubhaloSFRinHalfRad']
    sfr_values.append(sfr_value)
    print(f'Subhalo SFR at snap={snap}, id={i}: {sfr_value}')

df = pd.read_csv('ILL_347122.csv')
df['SFR'] = sfr_values
df.to_csv('ILL_347122.csv', index=False)

Subhalo SFR at snap=54, id=21803: 2.4654717445373535
Subhalo SFR at snap=60, id=37546: 29.487083435058594
Subhalo SFR at snap=68, id=81102: 7.017075061798096
Subhalo SFR at snap=85, id=197241: 2.5056843757629395
Subhalo SFR at snap=103, id=252489: 0.2741098999977112
Subhalo SFR at snap=135, id=347122: 0.2927980124950409


In [43]:
# Create empty list to store Eddington rate values
λ_rate_list = []

fields=['BH_Mass', 'BH_Mdot']

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    if blackholes and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = 0.0  # Set to np.nan so that the resulting λ_rate will be np.nan.
        
    if blackholes and 'BH_Mass' in blackholes and len(blackholes['BH_Mass']) > 0 and blackholes['BH_Mass'][0] != 0: 
        bh_mass = blackholes['BH_Mass'][0] * ((1e10* Msun) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0

    # Calculate L_bol and L_edd
    L_bol = e_r * bhdot * c**2
    
    L_edd = 4 * np.pi * G * bh_mass * m_p * c / sigma_T
        
    # Calculate λ_rate
    λ_rate = (L_bol / L_edd) if L_edd != 0 else 0

    # Append λ_rate to the list
    λ_rate_list.append(λ_rate)

# Print Eddington rate values
print("Eddington rate λ values", λ_rate_list)

df = pd.read_csv('ILL_347122.csv')
df['λ'] = λ_rate_list
df.to_csv('ILL_347122.csv', index=False)

Eddington rate λ values [0.6356833759363, 0.022874982159498916, 0.004935578305231227, 0.013674225393335165, 0.021897599501231824, 0.0008031817034028188]


In [44]:
# Create empty list to store the bolometric luminosity values
L_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting L_bol will be np.nan.

    # Calculate L_bol
    L_bol = e_r * bhdot * c**2

    
    # Append bolometric luminosity to the list
    L_list.append(np.log10(L_bol))


# Print bolometric luminosity values
print("Bol Luminosity values:")
print(L_list)

df = pd.read_csv('ILL_347122.csv')
df['L_bol'] = L_list
df.to_csv('ILL_347122.csv', index=False)

Bol Luminosity values:
[44.62638368290705, 44.11566434417578, 44.07278807689735, 44.60749985960331, 44.932973715735585, 43.601947811228555]


In [45]:
# Create empty list to store the Eddington luminosity values
L_edd_list = []

e_r = 0.2

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    if blackholes and 'BH_Mass' in blackholes and len(blackholes['BH_Mass']) > 0 and blackholes['BH_Mass'][0] != 0: 
        bh_mass = blackholes['BH_Mass'][0] * ((1e10* Msun) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0

    # Calculate L_edd
    L_edd = 4 * np.pi * G * bh_mass * m_p * c / sigma_T

    
    # Append Eddington luminosity to the list
    L_edd_list.append(np.log10(L_edd))


# Print Eddington luminosity values
print("Edd Luminosity values:")
print(L_edd_list)

df = pd.read_csv('ILL_347122.csv')
df['L_edd'] = L_edd_list
df.to_csv('ILL_347122.csv', index=False)

Edd Luminosity values:
[44.82314282877332, 45.756303580149165, 46.37945003031597, 46.47159712547011, 46.59257720731142, 46.697134004607015]


In [46]:
# Create empty list to store the feedback energy of the AGN values
E_feed_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for snapshot, id in zip(snapshots, ids):
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting feedback energy will be np.nan.

        
    # Calculate feedback energy
    E_feed = e_fm * e_r * bhdot * c**2

    
    # Append feedback energy to the list
    E_feed_list.append(np.log10(E_feed))


# Print feedback energy values
print("E_feed values:")
print(E_feed_list)

df = pd.read_csv('ILL_347122.csv')
df['E_feed'] = E_feed_list
df.to_csv('ILL_347122.csv', index=False)

E_feed values:
[43.92741367857103, 43.41669433983976, 43.373818072561335, 43.90852985526729, 44.234003711399566, 42.90297780689254]


In [47]:
#lets view the compilation of the file 
df = pd.read_csv('ILL_347122.csv')
print(df.head())

   Redshift (z)  Average divergence  Average radial velocity  \
0           4.0           -6.451352              -138.197074   
1           3.0           -3.934988               -39.687559   
2           2.0           -0.886356              -101.731036   
3           1.0           -4.319169               -27.106962   
4           0.5            8.728567                 6.576777   

   Blackhole Masses     Bhmdot        SFR         λ      L_bol      L_edd  \
0          6.723656  24.371712   2.465472  0.635683  44.626384  44.823143   
1          7.656817  23.860993  29.487083  0.022875  44.115664  45.756304   
2          8.279963  23.818117   7.017075  0.004936  44.072788  46.379450   
3          8.372110  24.352828   2.505684  0.013674  44.607500  46.471597   
4          8.493091  24.678302   0.274110  0.021898  44.932974  46.592577   

      E_feed  
0  43.927414  
1  43.416694  
2  43.373818  
3  43.908530  
4  44.234004  


---------------------

Test to see if we can mining values from all the snapshots between from 135 to 54

In [34]:
ids = [305959] #galaxy id
#target_snaps = [135, 103, 85, 68, 60, 54]  # list of target snapshots here
snapsh = list(range(135, 53, -1))
# the target snapshots list are the 7 out the 10 full snapshots that illustrisTNG have 
# 135 snapshots corresponds to z=0 and the 54 snapshot to z=4

progenitor_ids_list = []

for id in ids:
    id_progenitors = []
    start_url = "http://www.tng-project.org/api/Illustris-1/snapshots/z=0/subhalos/" + str(id)
    sub = get(start_url)
    current_snap = 135

    for snap in target_snaps:
        while current_snap > snap:
            # request the full subhalo details of the progenitor by following the sublink URL
            try:
                sub = get(sub['related']['sublink_progenitor'])
                current_snap = sub['snap']
            except KeyError:  # KeyError occurs if there's no progenitor
                break  # exit the while loop and go to the next snap

        if current_snap == snap:  # if the snapshot has a progenitor
            id_progenitors.append(sub['id'])
            print(f'Progenitor of {id} at snap={snap} is {sub["id"]}')
        else:
            print(f'Progenitor of {id} not followed to snap={snap}!')
            id_progenitors.append(-1)

    progenitor_ids_list.append(id_progenitors)

# Transpose the list to match the target_snap order
progenitor_ids_list = list(map(list, zip(*progenitor_ids_list)))

# Exclude -1 values
progenitor_ids_list = [sublist for sublist in progenitor_ids_list if sublist != [-1]*len(ids)]

# Flatten the list
progenitor_ids_list = [id for sublist in progenitor_ids_list for id in sublist]

# Reverse the list
progenitor_ids_list.reverse()

print(progenitor_ids_list)

Progenitor of 305959 at snap=135 is 305959
Progenitor of 305959 at snap=134 is 304327
Progenitor of 305959 at snap=133 is 300729
Progenitor of 305959 at snap=132 is 299560
Progenitor of 305959 at snap=131 is 295211
Progenitor of 305959 at snap=130 is 291414
Progenitor of 305959 at snap=129 is 287347
Progenitor of 305959 at snap=128 is 285031
Progenitor of 305959 at snap=127 is 282680
Progenitor of 305959 at snap=126 is 279968
Progenitor of 305959 at snap=125 is 277583
Progenitor of 305959 at snap=124 is 272905
Progenitor of 305959 at snap=123 is 270800
Progenitor of 305959 at snap=122 is 268684
Progenitor of 305959 at snap=121 is 259358
Progenitor of 305959 at snap=120 is 256401
Progenitor of 305959 at snap=119 is 252788
Progenitor of 305959 at snap=118 is 248332
Progenitor of 305959 at snap=117 is 239560
Progenitor of 305959 at snap=116 is 233940
Progenitor of 305959 at snap=115 is 231186
Progenitor of 305959 at snap=114 is 228616
Progenitor of 305959 at snap=113 is 225268
Progenitor 

In [18]:
def snapshot_to_redshift(snapshot):
    m = (4 - 0) / (54 - 135)  # Slope of the line
    return m * (snapshot - 135)

# This is your list of snapshots
target_snaps = list(range(135, 53, -1))

# Now, you can create a new list of redshifts corresponding to your list of snapshots
target_redshifts = [snapshot_to_redshift(snapshot) for snapshot in target_snaps]

target_redshifts.reverse()

print(target_redshifts)

[4.0, 3.950617283950617, 3.901234567901234, 3.8518518518518516, 3.802469135802469, 3.753086419753086, 3.7037037037037033, 3.654320987654321, 3.6049382716049383, 3.5555555555555554, 3.5061728395061724, 3.45679012345679, 3.4074074074074074, 3.3580246913580245, 3.3086419753086416, 3.259259259259259, 3.2098765432098766, 3.1604938271604937, 3.1111111111111107, 3.0617283950617282, 3.0123456790123457, 2.962962962962963, 2.91358024691358, 2.8641975308641974, 2.814814814814815, 2.765432098765432, 2.716049382716049, 2.6666666666666665, 2.617283950617284, 2.567901234567901, 2.518518518518518, 2.4691358024691357, 2.419753086419753, 2.3703703703703702, 2.3209876543209873, 2.271604938271605, 2.2222222222222223, 2.1728395061728394, 2.1234567901234565, 2.074074074074074, 2.0246913580246915, 1.9753086419753085, 1.9259259259259258, 1.876543209876543, 1.8271604938271604, 1.7777777777777777, 1.728395061728395, 1.6790123456790123, 1.6296296296296295, 1.5802469135802468, 1.5308641975308641, 1.48148148148148

In [19]:
# Create a DataFrame from your data
df = pd.DataFrame({
    'Redshift (z)': target_redshifts
})

# Write the DataFrame to a CSV file
df.to_csv('test.csv', index=False)

In [41]:
ids = progenitor_ids_list

In [40]:
snapsh = list(range(54, 136))

In [42]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for snap, id in zip(snapsh, ids):
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snap, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    print(f'Blackhole Masses at snap={snap}, id={id}: {bh_mass}')

Blackhole Masses at snap=54, id=55141: 5.524257081657102


KeyError: "Can't open attribute (can't locate attribute: 'FileOffsets_Snap')"

it is not possible to mine data for mini snapshots, because they do not have all the data that it needs